In [2]:
import pandas as pd
import re 
import spacy 
import os 
import numpy as np

In [77]:
# Load data 
eng_resumes = pd.read_csv("C:/Users/NIV/Documents/02-HR-project/05182020_english_resumes_V1.0.csv")

In [78]:
#!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

In [79]:
# Prepare all employee names
first_last = eng_resumes.employee_name.str.replace(",", "").apply(lambda x: x.split(" ")[1]+" "+x.split(" ")[0]).to_list()
last_first = eng_resumes.employee_name.str.replace(",", "").apply(lambda x: x.split(" ")[0]+" "+x.split(" ")[1]).to_list()
last = eng_resumes.employee_name.str.replace(",", "").apply(lambda x: (x.split(" ")[0])).to_list()
first = eng_resumes.employee_name.str.replace(",", "").apply(lambda x: x.split(" ")[1]).to_list()
last_upper = list(map(lambda x: x.upper(), last))
first_upper = list(map(lambda x: x.upper(), first))
middle = (eng_resumes.employee_name.str.replace(",", "")
     .apply(lambda x: x if len(x.split(" "))>2 else "")
     .where(lambda x: x!="")
     .dropna()
     .apply(lambda x: x.split(" ")[2])
     .to_list())
middle_upper = list(map(lambda x: x.upper(), middle))
names = first_last + last_first + last + first + middle + first_upper + last_upper + middle_upper
pat_names =r'\b(?:{})\b'.format('|'.join(names))

In [13]:
# Prepare all cities
f = open("cities.txt")
lines = f.readlines()
f.close()

cities = [re.match('[0-9]+\:\s(.+)\,\s(.+)', line).group(1) for line in lines]
provinces = [re.match('[0-9]+\:\s(.+)\,\s(.+)', line).group(2) for line in lines]
#cities = list(set(cities))
provinces = list(set(provinces)) 
abbre_prov = ["AB", "BC", "MB", "NB", "NL", "NT", "NS", "NU", "ON", "PE", "QC", "SK", "YT"]
provinces = provinces + abbre_prov

pat_cities =r'\b(?:{})\b'.format('|'.join(cities))
pat_provinces = r'\b(?:{})\b'.format('|'.join(provinces))

In [12]:
lines[:15]

['1: 100 Mile House, British Columbia\n',
 '2: 108 Mile House, British Columbia\n',
 '3: 108 Mile Ranch, British Columbia\n',
 '4: 150 Mile House, British Columbia\n',
 '5: Abbey, Saskatchewan\n',
 '6: Abbotsford, British Columbia\n',
 '7: Aberarder, Ontario\n',
 '8: Abercorn, Quebec\n',
 '9: Aberdeen, Saskatchewan\n',
 '10: Abernethy, Saskatchewan\n',
 '11: Abitibi Canyon, Ontario\n',
 '12: Acadia Valley, Alberta\n',
 '13: Acme, Alberta\n',
 '14: Acton, Ontario\n',
 '15: Acton Vale, Quebec\n']

In [19]:
cities

['100 Mile House',
 '108 Mile House',
 '108 Mile Ranch',
 '150 Mile House',
 'Abbey',
 'Abbotsford',
 'Aberarder',
 'Abercorn',
 'Aberdeen',
 'Abernethy',
 'Abitibi Canyon',
 'Acadia Valley',
 'Acme',
 'Acton',
 'Acton Vale',
 'Adamsville',
 'Adolphustown',
 'Advocate Harbour',
 'Agassiz',
 'Agassiz Provincial Forest',
 'Aguanish',
 'Ahousat',
 'Ailsa Craig',
 'Airdrie',
 'Ajax',
 'Aklavik',
 'Alameda',
 'Alban',
 'Albanel',
 'Albert',
 'Albert Mines',
 'Alberta Beach',
 'Alberton',
 'Alder Flats',
 'Aldergrove',
 'Alderville First Nation',
 'Alert Bay',
 'Alexander',
 'Alexandria',
 'Alexis Creek',
 'Alfred',
 'Algoma Mills',
 'Alida',
 'Alix',
 'Alkali Lake',
 'Allan',
 'Allardville',
 'Alliance',
 'Alliston',
 'Alma',
 'Alma',
 'Almonte',
 'Alonsa',
 'Alouette',
 'Alsask',
 'Altario',
 'Altona',
 'Alvena',
 'Alvinston',
 'Amherst',
 'Amherstburg',
 'Amos',
 'Amqui',
 'Ancaster',
 'Andrew',
 'Aneroid',
 'Angliers',
 'Angus',
 'Anjou',
 'Annapolis Royal',
 'Antigonish',
 'Anzac',
 'Ap

In [4]:
# Prepare otherwords 
f = open("otherwords.txt")
otherwords = [re.sub('\n', '', word) for word in f.readlines()]
f.close()

pat_otherwords = r'\b(?:{})\b'.format('|'.join(otherwords))

In [6]:
otherwords

['email',
 'Email',
 'E-mail',
 'Ave',
 'Profile',
 'North',
 'Northern',
 'South',
 'Southern',
 'East',
 'Eastern',
 'West',
 'Western',
 'St.Georges',
 'Cell',
 'Home',
 'Address',
 'Ridge',
 'Boulevard',
 'Canada',
 'page',
 'Page',
 'Stocker Rd',
 'Street',
 'Objective',
 'Highlights',
 'education ',
 'EDUCATION',
 'company',
 'CONTACT',
 'language',
 'phone',
 'Phone',
 'experience']

In [5]:
pat_otherwords

'\\b(?:email|Email|E-mail|Ave|Profile|North|Northern|South|Southern|East|Eastern|West|Western|St.Georges|Cell|Home|Address|Ridge|Boulevard|Canada|page|Page|Stocker Rd|Street|Objective|Highlights|education |EDUCATION|company|CONTACT|language|phone|Phone|experience)\\b'

In [3]:
os.getcwd()

'C:\\Users\\NIX\\Documents\\Project\\Github\\fork\\591_capstone_2020-glentel-mds\\src\\data_cleaning'

## Start preprocessing

In [82]:
def preprocess(text, min_token_len = 2, 
               irrelevant_pos = ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']):
    try:
        text = re.sub(r'([\n]+)', ' ', text)
        # Remove Emails
        #text = re.sub(r'\S*@\S*\s?', '', text)
        text = re.sub(r'([\w\.-]+@[\w\.-]+)', '', text)

        # Replace names, cities and provinces 
        text = re.sub(pat_names, '', text)

        text = re.sub(pat_cities, '', text)

        text = re.sub(pat_provinces, '', text)

        text = re.sub(pat_otherwords, '', text)

        # Replace a sequence of whitespaces by a single whitespace
        text = re.sub(r'\s+', ' ', text)

        # Remove other strange characters
        text = re.sub(r'''[\*\~]+''', "", text)

        doc = nlp(text)
        result = ' '.join([token.lemma_ for token in doc 
                      if token.is_stop == False
                          and token.pos_ not in irrelevant_pos 
                          and token.is_alpha 
                          and not token.like_url 
                          and not token.like_email 
                          and not token.is_stop
                          #and not token.text.isupper()
                          and not token.text.isdigit()
                          and not token.is_punct
                          and len(token) >= min_token_len])
    except TypeError:
        return("no resume found")
    return result

In [83]:
eng_resumes["clean_text"] = eng_resumes['resume_text'].apply(preprocess)

In [90]:
eng_resumes.to_csv("C:/Users/NIV/Documents/02-HR-project/05182020_cleaned_english_resumes_V1.1.csv")